# Data visualtisation Notebook

## Objectives

*   Answer business requirement 1: \n",
         * The client is interested to have a more efficient way to visually differentiate a healthy and powdery mildew infected leaves.

## Inputs

*   inputs/cherry_dataset/cherry-leaves/train
*   inputs/cherry_dataset/cherry-leaves/validation
*   inputs/cherry_dataset/cherry-leaves/test

## outputs

*   Image shape embedding file
*   Mean and variabillity of images per label plot
*   Plot to provide the contrast beetween healthy and powdery mildew leaves.
*   Generate code that awnsers the requirement 1 and can be used to build an image montage with the streamlit dashboard.

## Additional Comments | Insights | Conclusions

# Set Data Directory

## Import Libraries

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
sns.set_style("white")
from matplotlib.image import imread

## Set Working Directory

In [2]:
cwd= os.getcwd()

In [4]:
os.chdir('/workspace/cherry')
print("You set a new current directory")

You set a new current directory


In [5]:
work_dir = os.getcwd()
work_dir

'/workspace/cherry'

## Set Input Directories

* Set train, validation, test paths.

In [6]:
my_data_dir = 'inputs/cherry_dataset/cherry-leaves'
train_path = my_data_dir + '/train'
val_path = my_data_dir + '/validation'
test_path = my_data_dir + '/test'

## Set Output Directory

In [ ]:
version = 'v1'
file_path = f'outputs/{version}'

if 'outputs' in os.listdir(work_dir) and version in os.listdir(work_dir + '/outputs'):
    print('Old version is already available create a new version.')
    pass
else:
    os.makedirs(name=file_path)